In [0]:
from IPython import get_ipython


 # part 2: population and feature selection

 * load original, minimally processed, consolidated study data
 * select target population via screening and exclusion
 * select features including preliminary feature engineering
 * create a new csv to hold study data

In [0]:
get_ipython().run_line_magic('matplotlib', 'inline')
get_ipython().run_line_magic('reload_ext', 'autoreload')
get_ipython().run_line_magic('autoreload', '2')



In [0]:
#np.sum([83246 ,33888 ,7235 ,708,9 ,74 ,266 ,731] )



In [0]:
import pandas as pd
import numpy as np
import os



In [0]:
np.set_printoptions(threshold=50, edgeitems=20)



In [0]:
# Set ipython's max row display
pd.set_option('display.max_row', 100)

# Set iPython's max column display
pd.set_option('display.max_columns', 50)



In [0]:
PATH = 'study_data/'



In [0]:
df_main = pd.read_csv('mbsaqip_originals/all_years/all_years.csv', low_memory=False, index_col=0)



In [0]:
total_number_of_candidates_for_inclusion = len(df_main); total_number_of_candidates_for_inclusion


 set seed to make results reproducible

 * this is used for sampling uniform distributions for 2015 age variables, explained below and in our manuscript

In [0]:
np.random.seed(0)


 dict to hold reasons for exclusion for use in flow chart

In [0]:
exclusions = {}


 specify the outcomes and feature set to be used in predictive models

In [0]:
USE_VARS = ['LEAK', 
            'CLOT', 
            'SEX', 
            'race_PUF', 
            'hispanic', 
            'CPT', 
            'GERD', 
            'MOBILITY_DEVICE', 
            'HISTORY_DVT', 
            'MI_ALL_HISTORY', 
            'PTC', 
            'PCARD', 
            'HIP', 
            'HTN_MEDS', 
            'HYPERLIPIDEMIA', 
            'VENOUS_STASIS', 
            'DIALYSIS', 
            'RENAL_INSUFFICIENCY', 
            'THERAPEUTIC_ANTICOAGULATION', 
            'DIABETES', 
            'SMOKER', 
            'FUNSTATPRESURG', 
            'COPD', 
            'OXYGEN_DEPENDENT', 
            'HISTORY_PE', 
            'SLEEP_APNEA', 
            'CHRONIC_STEROIDS', 
            'IVCF', 
            'ASACLASS', 
            'ASSISTANT_TRAINING_LEVEL', 
            'OPYEAR', 
            'HYPOALB', 
            'ANEMIA', 
            'HGTCM', 
            'BMI_CONSOL', 
            'BMI_DELTA', 
            'WGTKG', 
            'OPLENGTH', 
            'AGE_CONSOL', 
            'ALBUMIN', 
            'HCT'
]


 the cpt codes we care about are:

 * lap gastric bypass
  * 43644
  * 43645

 * lap sleeve gastrectomy
  * 43775

In [0]:
include_cpts = [43644, 43645, 43775] 



In [0]:
df_exclude_cpts = df_main[~df_main['CPT'].isin(include_cpts)]
exclusions['wrong cpt'] = len(df_exclude_cpts)



In [0]:
df_main = df_main[df_main['CPT'].isin(include_cpts)]


 Alizadeh: "The study population also excluded patients who were under 18 years of age, undergoing revisional bariatric surgery, had previous bariatric or foregut surgery, and undergoing emergency surgery"

 exclude revisional cases

In [0]:
exclusions['revisional cases'] = df_main['CPTUNLISTED_REVCONV'].value_counts(dropna=False)[1]



In [0]:
df_main = df_main[df_main['CPTUNLISTED_REVCONV'] == 0]


 exclude patients with history of previous bariatric or foregut surgery

In [0]:
exclusions['previous obesity or foregut surgery'] = df_main['PREVIOUS_SURGERY'].value_counts(dropna=False)['Yes']



In [0]:
df_main = df_main[df_main['PREVIOUS_SURGERY'] == 'No']


 exclude children

In [0]:
df_kids = df_main[df_main['AGE'] < 18.0]
df_elderly = df_main[df_main['AGE'] >= 80.0]
df_confused = df_main[~((df_main['AGE'] >= 18.0) | (df_main['AGE'] < 18.0) | (df_main['AGE'] >= 80.0))]
df_main = df_main[(df_main['AGE'] >= 18.0) & (df_main['AGE'] < 80.0)]



In [0]:
exclusions['age under 18'] = len(df_kids)
exclusions['age 80 or over'] = len(df_elderly)
exclusions['age not available'] = len(df_confused)


 exclude emergency cases

In [0]:
exclusions['emergency surgery'] = len(df_main[df_main['PRIORITY'] == 'Yes'])



In [0]:
df_main = df_main[df_main['PRIORITY'] == 'No']



In [0]:
print(len(df_main))


 consolidate lap gastric bypass procedures

In [0]:
df_main['CPT'] = np.where(df_main['CPT'] == 43645, 43644, df_main['CPT'])


 there is no reason to consolidate age variables because the max age in the data is 80,  there is no patient without an age, and none of the remaining patients have an age greater than 80.

 for these reasons we can safely ignore the age > 80 variable as well

In [0]:
df_main.groupby('OPYEAR')['ageGT80'].value_counts(dropna=False)



In [0]:
print('the max age in the data is', max(df_main['AGE']))
print('there are', df_main['AGE'].isna().sum(), 'patients without an age')


 2016 and 2017 have precise age and 2015 has just age floor. options include: take the floor of all ages; do a 'half cycle correction' and take the midpoint of all ages. my preferred option is to draw uniform samples between age and age + 1 among patients in 2015. this minimizes the overall error in age without introducing unnatural patterns into the data. this allows us to cluster ages into categories later without any issues

In [0]:
df_main['AGE_CONSOL'] = np.where(df_main['OPYEAR'] == 2015, 
                                 df_main['AGE'] + np.random.uniform(0,1,1),
                                 df_main['AGE'])


 consolidate IVC filter variables

In [0]:
df_main['IVC_FILTER'].value_counts(dropna=False)#.sum()



In [0]:
df_main['IVC_TIMING'].value_counts(dropna=False)



In [0]:
df_main['IVCF'] = np.where(df_main['IVC_FILTER'] == 'Yes', df_main['IVC_TIMING'], df_main['IVC_FILTER'])


 dealing with missing height and weight data

 first, look to see if we can calculate BMI from weight in the event that BMI is not available (we can't)

In [0]:
# exclude cases with no information on BMI or weight
df_bmi_or_wgt_info = df_main[~(df_main['BMI_HIGH_BAR'].isna() & 
                df_main['WGT_HIGH_BAR'].isna() & 
                df_main['WGT_CLOSEST'].isna()  & 
                df_main['BMI'].isna())]

# exclude cases with no information on BMI  
df_bmi_info = df_main[~(df_main['BMI_HIGH_BAR'].isna() & 
                df_main['BMI'].isna())]



In [0]:
if len(df_bmi_or_wgt_info) == len(df_bmi_info):
    print('there are no missing BMI columns for which we can calculate BMI from weight')
    print('get rid of cases where there is no information on BMI or weight')
    print('we lose', len(df_main) - len(df_bmi_or_wgt_info), 'cases')
    exclusions['no info on bmi or weight'] = len(df_main) - len(df_bmi_or_wgt_info)



In [0]:
df_main = df_bmi_or_wgt_info.copy()


 next, look at cases with incomplete BMI information. For example, where most recent BMI is available but max BMI is not, or vice versa

In [0]:
df1 = df_main[df_main['BMI_HIGH_BAR'].isna() & (~df_main['BMI'].isna())]



In [0]:
df2 = df_main[df_main['BMI'].isna() & (~df_main['BMI_HIGH_BAR'].isna())]



In [0]:
print('there are', len(df1), 'cases where most recent BMI is available but max BMI is not')
print('there are', len(df2), 'cases where max BMI is available but most recent BMI is not')


 where most recent BMI is not available, assume max BMI

In [0]:
df_main['BMI_CONSOL'] = np.where(df_main['BMI'].isna(), df_main['BMI_HIGH_BAR'], df_main['BMI'])



In [0]:
df_main[['HGT', 'BMI_CONSOL']].isna().sum()


 BMI change (may be a useful feature)

In [0]:
df_main['BMI_DELTA'] = df_main['BMI_CONSOL'] - df_main['BMI_HIGH_BAR']


 in cases where we can't calculate the delta, we will assume no change. for now, leave as NaN so that we can build Table 1 in a subsequent notebook. NaNs will get set to the population average in a later processing step.

In [0]:
print('in', df_main['BMI_DELTA'].isna().sum(), 'cases, we cannot calculate the delta.')


 convert all heights to cm

In [0]:
df_main['HGTCM'] = np.where(df_main['HGTUNIT']=='cm', df_main['HGT'], df_main['HGT'] * 2.54)


 back-calculate weight from height and bmi

In [0]:
df_main['WGTKG'] = ((df_main['HGTCM']/100) ** 2) * df_main['BMI_CONSOL']


 looking at distribution of cases by year now that all exclusions have been applied

In [0]:
df_main.groupby('OPYEAR')['CPT'].value_counts(dropna=False)


 dealing with missing variables for albumin, hct, and asa class.

In [0]:
print('number of cases missing albumin:',df_main['ALBUMIN'].isna().sum())
print('number of cases missing hematocrit:',df_main['HCT'].isna().sum())
print('number of cases missing asa class:',df_main['ASACLASS'].isna().sum())


 albumin lowest quintile and distribution by year

In [0]:
df_main['ALBUMIN'].quantile(q=0.2,  interpolation='linear')



In [0]:
df_main['ALBUMIN'].hist(by=df_main['OPYEAR'], bins = 7)


 hct lowest quintile and distribution by year

In [0]:
df_main['HCT'].quantile(q=0.2,  interpolation='linear')



In [0]:
df_main['HCT'].hist(by=df_main['OPYEAR'], bins = 20)


 because there are so many missing hct and albumin values, it may make sense to experiment with treating them as categorical, in which case we could specify an 'unknown' category.

 similarly, since ASA class is categorical, we can specify an unknown category (or experiment with just getting rid of those cases in some models)

 create categories for anemia and hypoalbuminemia. for the former, we take the lowest quintile. for the latter, we take all albumin under 3.5 based on the methods from alizadeh

In [0]:
df_main['ANEMIA_temp'] = np.where(df_main['HCT'] < 38, 1, 2)
df_main['ANEMIA']      = np.where(df_main['HCT'].isna(), 0, df_main['ANEMIA_temp'])

df_main['HYPOALB_temp'] = np.where(df_main['ALBUMIN'] < 3.5, 1, 2)
df_main['HYPOALB']      = np.where(df_main['ALBUMIN'].isna(), 0, df_main['ANEMIA_temp'])


 make sure we have the correct number of missing anemia and hypoalbuminema values by ensuring they match the number of missing hct and alb values above

In [0]:
print(len(df_main[df_main['ANEMIA']==0]))
print(len(df_main[df_main['HYPOALB']==0]))


 convert null asa class values to separate 'unknown' category for easier data handling

In [0]:
df_main['ASACLASS'] = np.where(df_main['ASACLASS'].isna(), 'Unknown', df_main['ASACLASS'])


 composite endpoints for clot and leak as defined respectively in Dang et al and Alizadeh et al

In [0]:
rationale_clot = ['Vein Thrombosis Requiring Therapy', 'Pulmonary Embolism']
rationale_leak = ['Anastomotic/Staple Line Leak']



In [0]:
yes = ['Yes']



In [0]:
df_composite_clot = (df_main['SUSPREASON'].isin(rationale_clot) |
                df_main['REOP_SUSPECTED_REASON_BAR'].isin(rationale_clot) |  
                df_main['INTV_REASON_BAR'].isin(rationale_clot) |
                df_main['VEINTHROMBREQTER'].isin(yes) |
                df_main['DEATH_CAUSE_BAR'].isin(rationale_clot))

df_composite_leak = (df_main['SUSPREASON'].isin(rationale_leak) |
                df_main['REOP_SUSPECTED_REASON_BAR'].isin(rationale_leak) |  
                df_main['INTV_REASON_BAR'].isin(rationale_leak) |
                df_main['OSSIPATOS'].isin(yes) |
                df_main['DRAIN_PRESENT_30DAY_BAR'].isin(yes))



In [0]:
df_main['CLOT'] = df_composite_clot
df_main['LEAK'] = df_composite_leak



In [0]:
df_main['CLOT'].value_counts(dropna=False)[True]



In [0]:
df_main['LEAK'].value_counts(dropna=False)[True]


 select down to only the columns we may wish to include in building models

In [0]:
df_main = df_main[USE_VARS].copy()


 check for columns with missing values

In [0]:
df_main[df_main.columns].isna().sum()


 re-index

In [0]:
df_main = df_main.reset_index(drop=True) # `drop=True` drops the pre-existing, out of order index, which is included by default



In [0]:
exclusions


 confirm we tallied up all exclusions correctly:

In [0]:
if total_number_of_candidates_for_inclusion - sum(exclusions.values()) - len(df_main) == 0:
    print('this confirms that the total size of the database minus the size of the exclusions is equal to the size of the analysis cohort')


 save data

 as in part 1, this will throw an error if it runs after the '~/all_years' directory has been built; if that happens just delete the directory (or write some additional code for better file handling).

In [0]:
# make a dir to hold data from all years
os.mkdir(f'{PATH}')

#
df_main.to_csv(f'{PATH}/study_data.csv')


